In [1]:
import numpy as np

In [27]:
#features:
'''
acousticness,  danceability, duration_ms*, energy, instrumentalness, key*, liveness, loudness, mode, speechiness, tempo*,  
time_signature*, valence
'''
#https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-audio-features
def residual_square_sum(song1, song2):
    list_of_features = [scaled_squared_difference(song1.acousticness, song2.acousticness),
                       scaled_squared_difference(song1.danceability, song2.danceability),
                       scaled_squared_difference(song1.duration_ms, song2.duration_ms),
                       scaled_squared_difference(song1.energy, song2.energy),
                       scaled_squared_difference(song1.instrumentalness, song2.instrumentalness),
                       scaled_squared_difference_circular(song1.key, song2.key, range(12)),
                       scaled_squared_difference(song1.liveness, song2.liveness),
                       scaled_squared_difference(song1.mode, song2.mode),
                       scaled_squared_difference(song1.speechiness, song2.speechiness),
                       scaled_squared_difference(song1.tempo, song2.tempo),
                       scaled_squared_difference(song1.time_signature, song2.time_signature),
                       scaled_squared_difference(song1.valence, song2.valence)]
    return np.sum(list_of_features)
    

In [25]:
def scaled_squared_difference(num1, num2):
    maxnum = max(abs(num1), abs(num2))
    return (num1-num2 / maxnum) ** 2

def scaled_squared_difference_circular(num1, num2, circular_array):
    print(find_circular_diff(num1, num2, circular_array))
    return (find_circular_diff(num1, num2, circular_array) / 6) **2

1


0.027777777777777776

In [23]:
def find_circular_diff(num1, num2, circular_array):
    
    if num2 >= num1:
        p1 = num1
        p2 = num2
    else:
        p1 = num2
        p2 = num1
    
    dist1 = p2 - p1
    dist2 = p1 + len(circular_array) - p2 

    return min(dist1,dist2)


find_circular_diff(0, 11, range(12))

1

In [71]:
lst = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l"]
npar2 = np.array(lst)

npar = np.array([9, 4, 4, 3, 3, 9, 0, 4, 6, 0, -2])
n_lowest = np.argpartition(npar, 4)[:4]
print(n_lowest, npar[n_lowest])
x = n_lowest[np.argsort(npar[n_lowest])]

npar2[x]

[10  9  6  4] [-2  0  0  3]


array(['k', 'j', 'g', 'e'], dtype='<U1')

In [2]:
import requests

client_id = '4e2ba88cc79247a8acc4160d1510764f'
client_secret ='1698c2d495ed4d6fba3967974ced8423'

AUTH_URL = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})
 # convert the response to JSON
auth_response_data = auth_response.json()
# save the access token
access_token = auth_response_data['access_token']

# populating the database with a playlist of the most popular songs
url = 'https://api.spotify.com/v1/playlists/'
playlist_id = '2YRe7HRKNRvXdJBp9nXFza' # 500 most popular Spotify songs
# playlist_id = '37i9dQZEVXbLRQDuF5jeBp' # Top 50 in US
queryparam = '?offset='
offset = 0
head = {'Authorization': 'Bearer ' + access_token}

list_of_tracks = []

while offset < 600:
    song_json = requests.get(url + playlist_id + '/tracks' + queryparam + str(offset), headers=head)
    print(.2 * offset/100)
    list_of_tracks += song_json.json().get('items')
    offset += 100

0.0
0.2
0.4
0.6
0.8
1.0


In [11]:
audio_url = 'https://api.spotify.com/v1/audio-features/'
list_of_songs = []
import time
for t in list_of_tracks:
    
    track = t.get('track')
    album = track.get('album')

    track_name = track.get('name')
    track_id = track.get('id')
    img_link = album.get('images')[0].get('url')
    track_album = album.get('name')
    artist_id = album.get('artists')[0].get('id')
    artist_name = album.get('artists')[0].get('name')

    # set the audio features of this song
    song_dict = {}
    song_dict['id'] = track_id
    song_dict['title'] = track_name
    song_dict['artist_id'] = artist_id
    song_dict['artist_name'] = artist_name
    song_dict['img_link'] = img_link
    song_dict['album'] = track_album
    audio_features = requests.get(audio_url + track_id, headers=head)
    print(audio_features.json())
    song_dict.update(audio_features.json())
    song = song_dict

    # song = Song(track_id, track_name, artist_id, artist_name, img_link, track_album)
    # audio_features = requests.get(audio_url + track_id, headers=head)
    # song.set_audio_features(audio_features.json())

    list_of_songs.append(song)

{'danceability': 0.825, 'energy': 0.652, 'key': 1, 'loudness': -3.183, 'mode': 0, 'speechiness': 0.0802, 'acousticness': 0.581, 'instrumentalness': 0, 'liveness': 0.0931, 'valence': 0.931, 'tempo': 95.977, 'type': 'audio_features', 'id': '7qiZfU4dY1lWllzX7mPBI3', 'uri': 'spotify:track:7qiZfU4dY1lWllzX7mPBI3', 'track_href': 'https://api.spotify.com/v1/tracks/7qiZfU4dY1lWllzX7mPBI3', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7qiZfU4dY1lWllzX7mPBI3', 'duration_ms': 233713, 'time_signature': 4}
{'danceability': 0.585, 'energy': 0.52, 'key': 5, 'loudness': -6.136, 'mode': 0, 'speechiness': 0.0712, 'acousticness': 0.124, 'instrumentalness': 7.01e-05, 'liveness': 0.131, 'valence': 0.129, 'tempo': 159.801, 'type': 'audio_features', 'id': '0e7ipj03S05BNilyu5bRzt', 'uri': 'spotify:track:0e7ipj03S05BNilyu5bRzt', 'track_href': 'https://api.spotify.com/v1/tracks/0e7ipj03S05BNilyu5bRzt', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0e7ipj03S05BNilyu5bRzt', 'duration_ms

{'danceability': 0.614, 'energy': 0.379, 'key': 4, 'loudness': -10.48, 'mode': 1, 'speechiness': 0.0476, 'acousticness': 0.607, 'instrumentalness': 0.000464, 'liveness': 0.0986, 'valence': 0.201, 'tempo': 107.989, 'type': 'audio_features', 'id': '1HNkqx9Ahdgi1Ixy2xkKkL', 'uri': 'spotify:track:1HNkqx9Ahdgi1Ixy2xkKkL', 'track_href': 'https://api.spotify.com/v1/tracks/1HNkqx9Ahdgi1Ixy2xkKkL', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1HNkqx9Ahdgi1Ixy2xkKkL', 'duration_ms': 258987, 'time_signature': 4}
{'danceability': 0.511, 'energy': 0.566, 'key': 6, 'loudness': -7.23, 'mode': 0, 'speechiness': 0.2, 'acousticness': 0.349, 'instrumentalness': 0, 'liveness': 0.34, 'valence': 0.218, 'tempo': 83.903, 'type': 'audio_features', 'id': '0s3nnoMeVWz3989MkNQiRf', 'uri': 'spotify:track:0s3nnoMeVWz3989MkNQiRf', 'track_href': 'https://api.spotify.com/v1/tracks/0s3nnoMeVWz3989MkNQiRf', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0s3nnoMeVWz3989MkNQiRf', 'duration_ms': 2

{'danceability': 0.687, 'energy': 0.792, 'key': 5, 'loudness': -2.749, 'mode': 1, 'speechiness': 0.0452, 'acousticness': 0.191, 'instrumentalness': 0, 'liveness': 0.167, 'valence': 0.671, 'tempo': 100.015, 'type': 'audio_features', 'id': '2dpaYNEQHiRxtZbfNsse99', 'uri': 'spotify:track:2dpaYNEQHiRxtZbfNsse99', 'track_href': 'https://api.spotify.com/v1/tracks/2dpaYNEQHiRxtZbfNsse99', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2dpaYNEQHiRxtZbfNsse99', 'duration_ms': 214290, 'time_signature': 4}
{'danceability': 0.572, 'energy': 0.385, 'key': 7, 'loudness': -6.362, 'mode': 1, 'speechiness': 0.0308, 'acousticness': 0.371, 'instrumentalness': 0, 'liveness': 0.231, 'valence': 0.323, 'tempo': 95.799, 'type': 'audio_features', 'id': '2VxeLyX666F8uXCJ0dZF8B', 'uri': 'spotify:track:2VxeLyX666F8uXCJ0dZF8B', 'track_href': 'https://api.spotify.com/v1/tracks/2VxeLyX666F8uXCJ0dZF8B', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2VxeLyX666F8uXCJ0dZF8B', 'duration_ms': 2157

{'danceability': 0.856, 'energy': 0.609, 'key': 0, 'loudness': -7.223, 'mode': 1, 'speechiness': 0.0824, 'acousticness': 0.00801, 'instrumentalness': 8.15e-05, 'liveness': 0.0344, 'valence': 0.928, 'tempo': 114.988, 'type': 'audio_features', 'id': '32OlwWuMpZ6b0aN2RZOeMS', 'uri': 'spotify:track:32OlwWuMpZ6b0aN2RZOeMS', 'track_href': 'https://api.spotify.com/v1/tracks/32OlwWuMpZ6b0aN2RZOeMS', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/32OlwWuMpZ6b0aN2RZOeMS', 'duration_ms': 269667, 'time_signature': 4}
{'danceability': 0.726, 'energy': 0.769, 'key': 6, 'loudness': -5.043, 'mode': 1, 'speechiness': 0.123, 'acousticness': 0.0293, 'instrumentalness': 0.0101, 'liveness': 0.104, 'valence': 0.733, 'tempo': 97.985, 'type': 'audio_features', 'id': '3B54sVLJ402zGa6Xm4YGNe', 'uri': 'spotify:track:3B54sVLJ402zGa6Xm4YGNe', 'track_href': 'https://api.spotify.com/v1/tracks/3B54sVLJ402zGa6Xm4YGNe', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3B54sVLJ402zGa6Xm4YGNe', 'dur

{'danceability': 0.746, 'energy': 0.754, 'key': 1, 'loudness': -6.684, 'mode': 1, 'speechiness': 0.0676, 'acousticness': 0.0152, 'instrumentalness': 0, 'liveness': 0.0486, 'valence': 0.746, 'tempo': 149.882, 'type': 'audio_features', 'id': '3zkWCteF82vJwv0hRLba76', 'uri': 'spotify:track:3zkWCteF82vJwv0hRLba76', 'track_href': 'https://api.spotify.com/v1/tracks/3zkWCteF82vJwv0hRLba76', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3zkWCteF82vJwv0hRLba76', 'duration_ms': 206880, 'time_signature': 4}
{'danceability': 0.845, 'energy': 0.567, 'key': 5, 'loudness': -8.118, 'mode': 0, 'speechiness': 0.0956, 'acousticness': 0.59, 'instrumentalness': 0.00142, 'liveness': 0.0811, 'valence': 0.793, 'tempo': 125.02, 'type': 'audio_features', 'id': '4B0JvthVoAAuygILe3n4Bs', 'uri': 'spotify:track:4B0JvthVoAAuygILe3n4Bs', 'track_href': 'https://api.spotify.com/v1/tracks/4B0JvthVoAAuygILe3n4Bs', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4B0JvthVoAAuygILe3n4Bs', 'duration_m

...

{'danceability': 0.826, 'energy': 0.773, 'key': 11, 'loudness': -4.218, 'mode': 0, 'speechiness': 0.0524, 'acousticness': 0.323, 'instrumentalness': 0, 'liveness': 0.143, 'valence': 0.761, 'tempo': 126.899, 'type': 'audio_features', 'id': '5iwz1NiezX7WWjnCgY5TH4', 'uri': 'spotify:track:5iwz1NiezX7WWjnCgY5TH4', 'track_href': 'https://api.spotify.com/v1/tracks/5iwz1NiezX7WWjnCgY5TH4', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5iwz1NiezX7WWjnCgY5TH4', 'duration_ms': 238200, 'time_signature': 4}
{'danceability': 0.866, 'energy': 0.613, 'key': 0, 'loudness': -6.385, 'mode': 1, 'speechiness': 0.151, 'acousticness': 0.0293, 'instrumentalness': 0, 'liveness': 0.104, 'valence': 0.359, 'tempo': 98.001, 'type': 'audio_features', 'id': '2toVe5hfuIi97ytDPDbQFt', 'uri': 'spotify:track:2toVe5hfuIi97ytDPDbQFt', 'track_href': 'https://api.spotify.com/v1/tracks/2toVe5hfuIi97ytDPDbQFt', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2toVe5hfuIi97ytDPDbQFt', 'duration_ms': 229

{'danceability': 0.843, 'energy': 0.391, 'key': 2, 'loudness': -7.899, 'mode': 1, 'speechiness': 0.0845, 'acousticness': 0.181, 'instrumentalness': 0, 'liveness': 0.137, 'valence': 0.496, 'tempo': 129.972, 'type': 'audio_features', 'id': '51Fjme0JiitpyXKuyQiCDo', 'uri': 'spotify:track:51Fjme0JiitpyXKuyQiCDo', 'track_href': 'https://api.spotify.com/v1/tracks/51Fjme0JiitpyXKuyQiCDo', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/51Fjme0JiitpyXKuyQiCDo', 'duration_ms': 160627, 'time_signature': 4}
{'danceability': 0.677, 'energy': 0.744, 'key': 10, 'loudness': -6.806, 'mode': 0, 'speechiness': 0.0295, 'acousticness': 0.0404, 'instrumentalness': 0.00016, 'liveness': 0.074, 'valence': 0.631, 'tempo': 124.08, 'type': 'audio_features', 'id': '1DFD5Fotzgn6yYXkYsKiGs', 'uri': 'spotify:track:1DFD5Fotzgn6yYXkYsKiGs', 'track_href': 'https://api.spotify.com/v1/tracks/1DFD5Fotzgn6yYXkYsKiGs', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1DFD5Fotzgn6yYXkYsKiGs', 'duration_m

{'danceability': 0.729, 'energy': 0.675, 'key': 8, 'loudness': -6.003, 'mode': 1, 'speechiness': 0.0312, 'acousticness': 0.175, 'instrumentalness': 1.58e-06, 'liveness': 0.55, 'valence': 0.779, 'tempo': 119.968, 'type': 'audio_features', 'id': '1fidCEsYlaVE3pHwKCvpFZ', 'uri': 'spotify:track:1fidCEsYlaVE3pHwKCvpFZ', 'track_href': 'https://api.spotify.com/v1/tracks/1fidCEsYlaVE3pHwKCvpFZ', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1fidCEsYlaVE3pHwKCvpFZ', 'duration_ms': 245867, 'time_signature': 4}
{'danceability': 0.525, 'energy': 0.433, 'key': 6, 'loudness': -10.598, 'mode': 1, 'speechiness': 0.185, 'acousticness': 0.107, 'instrumentalness': 0, 'liveness': 0.135, 'valence': 0.276, 'tempo': 160.108, 'type': 'audio_features', 'id': '43PuMrRfbyyuz4QpZ3oAwN', 'uri': 'spotify:track:43PuMrRfbyyuz4QpZ3oAwN', 'track_href': 'https://api.spotify.com/v1/tracks/43PuMrRfbyyuz4QpZ3oAwN', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/43PuMrRfbyyuz4QpZ3oAwN', 'duration_ms

{'danceability': 0.605, 'energy': 0.822, 'key': 0, 'loudness': -4.833, 'mode': 1, 'speechiness': 0.0438, 'acousticness': 0.00671, 'instrumentalness': 0.134, 'liveness': 0.147, 'valence': 0.288, 'tempo': 167.997, 'type': 'audio_features', 'id': '1zB4vmk8tFRmM9UULNzbLB', 'uri': 'spotify:track:1zB4vmk8tFRmM9UULNzbLB', 'track_href': 'https://api.spotify.com/v1/tracks/1zB4vmk8tFRmM9UULNzbLB', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1zB4vmk8tFRmM9UULNzbLB', 'duration_ms': 187147, 'time_signature': 4}
{'danceability': 0.383, 'energy': 0.637, 'key': 3, 'loudness': -6.993, 'mode': 1, 'speechiness': 0.0703, 'acousticness': 0.0722, 'instrumentalness': 0, 'liveness': 0.368, 'valence': 0.0827, 'tempo': 177.913, 'type': 'audio_features', 'id': '2RSHsoi04658QL5xgQVov3', 'uri': 'spotify:track:2RSHsoi04658QL5xgQVov3', 'track_href': 'https://api.spotify.com/v1/tracks/2RSHsoi04658QL5xgQVov3', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2RSHsoi04658QL5xgQVov3', 'duration_

{'danceability': 0.497, 'energy': 0.693, 'key': 8, 'loudness': -7.024, 'mode': 1, 'speechiness': 0.0565, 'acousticness': 0.19, 'instrumentalness': 0, 'liveness': 0.215, 'valence': 0.358, 'tempo': 111.539, 'type': 'audio_features', 'id': '6cFZ4PLC19taNlpl9pbGMf', 'uri': 'spotify:track:6cFZ4PLC19taNlpl9pbGMf', 'track_href': 'https://api.spotify.com/v1/tracks/6cFZ4PLC19taNlpl9pbGMf', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6cFZ4PLC19taNlpl9pbGMf', 'duration_ms': 296493, 'time_signature': 3}
{'danceability': 0.39, 'energy': 0.649, 'key': 2, 'loudness': -5.163, 'mode': 1, 'speechiness': 0.0345, 'acousticness': 0.172, 'instrumentalness': 0, 'liveness': 0.216, 'valence': 0.144, 'tempo': 121.16, 'type': 'audio_features', 'id': '6lEuFGrUqR9Yc6hSsHPnYP', 'uri': 'spotify:track:6lEuFGrUqR9Yc6hSsHPnYP', 'track_href': 'https://api.spotify.com/v1/tracks/6lEuFGrUqR9Yc6hSsHPnYP', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6lEuFGrUqR9Yc6hSsHPnYP', 'duration_ms': 299760